# CNN Multichannel
A notebook to study the importance of features from sEMG measurements when explaining hand movements. Explained through the use of CNN and Saliency maps.

In [17]:
# pytorch will be our main engine for this project
import torch

# helper library for sEMG processing
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

# visualisation tools
import torchvision
from torchview import draw_graph

In [18]:
# Seed for reproducibility
seed_value = 201700448
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import torch
torch.manual_seed(seed_value)
if torch.backends.mps.is_available():
    torch.mps.manual_seed(seed_value)

### Step 1: import data

In [19]:
ds = torch.load('dataset_10-19.pt')
# explain the dataset
print(ds)

tensor([[-5.0040e-07,  1.4964e-06, -1.4122e-05,  ...,  8.9008e-07,
          6.5215e-06,  1.0000e+00],
        [ 1.7071e-07,  3.3427e-06, -1.3787e-05,  ...,  2.5688e-06,
          4.7953e-06,  1.0000e+00],
        [ 8.4182e-07,  4.0140e-06, -1.2612e-05,  ..., -1.1656e-07,
          1.6840e-06,  1.0000e+00],
        ...,
        [ 7.5726e-07, -1.7326e-06, -5.8029e-06,  ..., -4.2426e-06,
         -1.3934e-06,  0.0000e+00],
        [ 2.6038e-06,  2.7994e-06, -3.4529e-06,  ..., -4.5785e-06,
         -1.8521e-06,  0.0000e+00],
        [ 3.2752e-06,  7.3314e-06,  1.2470e-06,  ..., -2.3966e-06,
          5.1100e-08,  0.0000e+00]])


### Step 2: prepare the data for CNN

In [4]:
# split the data into train, test and validation
train_size = int(0.8 * len(ds))
test_size = int(0.1 * len(ds))
val_size = len(ds) - train_size - test_size

train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(ds, [train_size, test_size, val_size])

print(f"Train: {len(train_dataset)}")
print(f"Test: {len(test_dataset)}")
print(f"Validation: {len(val_dataset)}")



Train: 70271699
Test: 8783962
Validation: 8783963


In [20]:
from torch.utils.data import Dataset, DataLoader

class sEMGDataset(Dataset):
    def __init__(self, ds):
        # Separate the sEMG data from the labels
        self.data = ds[:, :12]  # First 12 columns are sEMG data
        self.labels = ds[:, 12].long()  # Last column is the label, converted to long for CrossEntropyLoss
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the sEMG data sample and reshape it to 12x1
        sample = self.data[idx].view(12,1)  # Reshape to 12 channels, 1 time step
        # we do this because the Conv1d layer in PyTorch expects the input to be in the format (batch, channels, time) 
        # Get the label
        label = self.labels[idx]
        return sample, label
    
    def to(self, mps_device):
        self.data = self.data.to(mps_device)
        self.labels = self.labels.to(mps_device)
        print("Dataset moved to", mps_device)


class DataLoaderX(DataLoader):
    def __init__(self, *args, **kwargs):
        super(DataLoaderX, self).__init__(*args, **kwargs)

    def to(self, mps_device):
        for batch_idx, (data, target) in enumerate(self):
            self.data[batch_idx] = data.to(mps_device)
            self.target[batch_idx] = target.to(mps_device)
        print("DataLoader moved to", mps_device)

### Step 3: Define the CNN model which will be used for training

In [7]:
import torch.nn as nn

class Conv1d_layer(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
        '''
        in_channels: number of input channels
        out_channels: number of output channels
        kernel_size: size of the kernel
        stride: stride of the kernel
        padding: padding of the kernel
        '''
        super(Conv1d_layer, self).__init__()
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.BatchNorm = nn.BatchNorm1d(out_channels)
        self.MaxPool1d = nn.MaxPool1d(kernel_size=2, stride=2)
    
    def forward(self, x):
        x = self.conv1d(x)
        x = self.BatchNorm(x)
        x = torch.relu(x)
        x = self.MaxPool1d(x)

        return x

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Conv1d_Net(nn.Module):
    def __init__(self, input_shape, kernel_size=3, stride=1, padding=1, out_channels=11):
        super(Conv1d_Net, self).__init__()
        in_channels = input_shape[1]  # Assuming input_shape is [channels, length]
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride, padding)
        self.conv3 = nn.Conv1d(out_channels, out_channels, kernel_size, stride, padding)
        # Removed reference to conv4 since it's not defined
        # Calculate the size of the output from conv layers dynamically
        self.fc1 = nn.Linear(self._get_conv_output(input_shape), 32)
        self.fc2 = nn.Linear(32, 10)
        # Removed softmax for reasons mentioned above

    def _forward_features(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x
    
    def _get_conv_output(self, shape):
        with torch.no_grad():  # Temporarily set all requires_grad flags to False
            input = torch.autograd.Variable(torch.rand(1, *shape))  # Dummy input
            output_feat = self._forward_features(input)
            n_size = output_feat.data.view(1, -1).size(1)
        return n_size
    
    def forward(self, x):
        x = self._forward_features(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the FC layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # Return logits directly if using nn.CrossEntropyLoss
        return x

    

In [34]:
ds_train = sEMGDataset(train_dataset.dataset)
dl_train = DataLoaderX(ds_train, batch_size=32, shuffle=False)

In [35]:
# plot dataset shape
print(ds_train.data.shape)

torch.Size([87839624, 12])


In [36]:
# plot dataloader first batch shape
print(next(iter(dl_train))[0].shape)

torch.Size([32, 12, 1])


In [41]:
import torch
import torchviz
from torch.nn import Conv1d, Linear, ReLU, Softmax  # Ensure you have the correct imports for your model

# Assuming Conv1d_Net is defined as per previous discussions
# Assuming dl_train is your DataLoader

device = 'cpu'
# Get the first batch from the DataLoader and prepare a single sample's shape
input, _ = next(iter(dl_train))  # Gets the first batch (input and labels)
input = input.to(device)  # Ensure input is on the right device

# The model expects the input shape without the batch size
# For Conv1d, the input shape should be (channels, length), excluding the batch size
# Since your DataLoader batch is shaped as [batch_size, channels, length], use one sample's shape for model initialization
sample_shape = input.shape[1:]  # This excludes the batch dimension

# Create an instance of the model with the correct input shape
model = Conv1d_Net(input_shape=sample_shape).to(device)

# Use torchviz to visualize the model
# The model(input) call should be wrapped in torchviz.make_dot
model_output = model(input)  # Forward pass with the corrected input
visual_graph = torchviz.make_dot(model_output, params=dict(model.named_parameters()))

# To render and view the graph, depending on your environment, you might directly display visual_graph
# or save it to a file. For example:
visual_graph.render("model_visualization", format="png", directory="/mnt/data")


RuntimeError: Given groups=1, weight of size [11, 1, 3], expected input[1, 12, 1] to have 1 channels, but got 12 channels instead

### STEP 4: Define the dataset and the dataloader, and draw the diagram of the model

In [8]:
# device agnostic code (will run on GPU if available, otherwise CPU)
device = torch.device('mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu'))

In [9]:
ds_train.to(torch.device(device = device))
print("Train shape: ", ds_train.data.shape)

Dataset moved to mps
Train shape:  torch.Size([87839624, 12])


### STEP 5: Start train/validation loop

In [16]:
learning_rate = 0.001
epochs = 1000

In [15]:
# Create the DataLoader
train_loader = DataLoaderX(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoaderX(val_dataset, batch_size=32, shuffle=False)  # No need to shuffle the validation data


In [17]:
# optimizer and loss function
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [19]:
# training loop
from tqdm import tqdm
import time

train_loss = []
val_loss = []



0it [00:00, ?it/s]


ValueError: too many values to unpack (expected 2)